<a href="https://colab.research.google.com/github/NganTran-0017/HIDS/blob/main/HIDS_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Update remove duplicate between train and test --> create independent test
check why test set after removing the intersection (aka independent test) has no frequent record? --> no, set(independent test) has fewer records than independent test.
So the previous clean_data method removes duplication and overlap instances, while the new method (remove_duplicate) only removes overlap instance


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score, roc_curve, auc, recall_score, precision_score,plot_confusion_matrix
from sklearn.model_selection import train_test_split
%matplotlib inline

import nltk
nltk.download("popular")

## Tokenizing syscall sequences into n-grams of 6

from nltk.tokenize import word_tokenize
from nltk import ngrams

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

#**Processing data**

Use the given datasets in our GitHub to load the data

## **ADFA-LD**

In [ ]:
# Load ADFA-LD and unzip them
! wget 'https://github.com/NganTran-0017/HIDS/blob/main/Datasets/ADFA-LD/Training_Data_Master.zip?raw=true'
! wget 'https://github.com/NganTran-0017/HIDS/blob/main/Datasets/ADFA-LD/Attack_Data_Master.zip?raw=true'
# Rename files
! mv Attack_Data_Master.zip?raw=true   Attack_Data_Master.zip
! mv Training_Data_Master.zip?raw=true Training_Data_Master.zip

# Unzip files
! unzip Training_Data_Master.zip
! unzip Attack_Data_Master.zip

from glob import glob
import os
train_dir = 'Training_Data_Master/'
test_dir  = 'Attack_Data_Master/'

# Read data in a dataframe
def read_in_data(path, is_normal = 1): 
  lines = []
  if is_normal == 1:
    for filename in os.listdir(path): # iterate through all label folders in a directory 
      with open(path+filename) as f:
        lines.append(f.read())
       # data = pd.read_csv(path+filename, sep=' ', header=None)
       # list_of_dataframes.append(data.stack())
    
  else:
    for folder in os.listdir(path): # iterate through different attack folders in a directory
      files = glob(str(path + folder +"/*.txt"), recursive=False) # get a list of files from each label folder 
      for filename in files: 
        with open(filename) as f:
          lines.append(f.read())
        #data = pd.read_csv(filename, sep=' ', header=None)
        #lines.append(data.stack())

  df = pd.DataFrame(lines)
  return df   


# Trainning set
df = read_in_data(train_dir)
df.rename(columns={0:'Syscall Sequence'}, inplace=True)

print('Normal training data size:', df.shape)
print(df.head(5))

# Testing set
intrusiondf = read_in_data(test_dir, is_normal = 0)
intrusiondf.rename(columns={0:'Syscall Sequence'}, inplace=True)

print('Intrusion Testing data size:', intrusiondf.shape)
intrusiondf.head(5)

## **UNM and MIT Lpr**

In [79]:
  ## Uncomment each line to load Normal data

# Synthetic sendmail csv_file = ['Sendmail-normal.txt']; DATA = 'Synthetic Sendmail'; DATA_I='Synthetic_Sendmail'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Sendmail-normal.txt'

# LIVE LPR csv_file = ['lpr-normal-all.txt']; DATA = 'Live Lpr'; DATA_I='Live-Lpr' 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/4.%20Live%20lpr/lpr-normal-all.txt'

# MIT live lpr csv_file = [ 'MIT-lpr-normal-all.txt']; DATA = 'MIT Live Lpr'; DATA_I='MIT-Lpr' 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/MIT/MIT-lpr-normal-all.txt'

# LOGIN and PS csv_file = [ 'login-normal.txt', 'ps-normal.txt']; DATA = 'Login and Ps'; DATA_I ="Login-and-Ps"
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/login-normal.txt'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/ps-normal.txt'

# INETD csv_file = [ 'inetd-live-unm.int']; DATA = 'Inetd';DATA_I ="Inetd" 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/8.Inetd/inetd-live-unm.int'

# STIDE csv_file = [ 'stide-normal.txt']; DATA = 'Stide';DATA_I ='Stide' 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/10.Stide/stide-normal.txt.gz'
#!gunzip 'stide-normal.txt.gz'

# Live Named csv_file = [ 'named-live-unm']; DATA = 'Live Named';DATA_I ='Live-Named' 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/6.Live_named/named-live-unm'

# Xlock csv_file = [ 'normal-xlock.txt']; DATA = 'Xlock';DATA_I='Xlock' 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/5.xlock/normal-xlock.txt'

# Synthetic Ftp csv_file = [ 'Syn-ftp-normal-all.txt']; DATA = 'Synthetic Ftp'; DATA_I='Synthetic-Ftp'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/2.Synthetic%20Ftp/Syn-ftp-normal-all.txt'

# Synthetic lpr 
csv_file = ['syn.int']; DATA = 'Synthetic Lpr';DATA_I='Synthetic-Lpr'
!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/3.Synthetic-lpr/syn.int'

# Concat a list of files into normal df
list_of_dataframes = []
for filename in csv_file:
    list_of_dataframes.append(pd.read_csv(filename, sep=' ', header=None, engine='python'))
df = pd.concat(list_of_dataframes)

# Check number of columns, if > 2, then drop the excess
if len(df.columns) > 2:
    df=df.drop(labels=None, axis=1, columns = [2,3])
df =df.rename(columns= {0:"PID", 1:"Syscall"})

print('Normal data size:', df.shape)


  ## Uncomment each line to load Intrusion data:

# Synthetic sendmail csv_file = ['Sendmail-intrusion.txt']
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/intrusion/Sendmail-intrusion.txt'

# LIVE LPR csv_file =['exploit-unm.int'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/4.%20Live%20lpr/Intrusion/exploit-unm.int'

# MIT live lpr csv_file = [ 'exploit-ai.int'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/MIT/Intrusion/exploit-ai.int'

# LOGIN and PS csv_file = [ 'login-homegrown.int','ps-homegrown.int','login-recovered.int','ps-recovered.int']
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/intrusion/ps-recovered.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/intrusion/ps-homegrown.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/intrusion/login-recovered.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/intrusion/login-homegrown.int'

# INETD csv_file = [ 'inetd-intrusion.int'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/8.Inetd/intrusion/inetd-intrusion.int'

# STIDE csv_file = [ 'stide-intrusion'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/10.Stide/intrusion/stide-intrusion'

# Live Named  csv_file = [ 'exploit-1.int','exploit-2.int'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/6.Live_named/intrusion/exploit-1.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/6.Live_named/intrusion/exploit-2.int'

# Xlock csv_file = [ 'xlock-intrusion-all.txt']
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/5.xlock/intrusion/xlock-intrusion-all.txt'

# Synthetic Ftp csv_file = [ 'exploit2.int']
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/2.Synthetic%20Ftp/intrusion/exploit2.int'

# Synthetic Lpr 
csv_file = [ 'exploit-unm.int']
!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/3.Synthetic-lpr/intrusion/exploit-unm.int'

list_of_dataframes = []
for filename in csv_file:
    list_of_dataframes.append(pd.read_csv(filename, sep=' ', header=None, engine='python'))
intrusiondf = pd.concat(list_of_dataframes)

if len(intrusiondf.columns) > 2:
    intrusiondf = intrusiondf.drop(labels=None, axis=1, columns = [2,3])
intrusiondf = intrusiondf.rename(columns= {0:"PID", 1:"Syscall"})

print('intrusion data size: ', intrusiondf.shape)

print('Normal:',df.head(3))
#print(df['PID'].value_counts())
print('Intrusion:',intrusiondf.head())

--2022-04-04 23:03:11--  https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/3.Synthetic-lpr/syn.int
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23144 (23K) [text/plain]
Saving to: ‘syn.int’

syn.int             100%[===================>]  22.60K  --.-KB/s    in 0.002s  

2022-04-04 23:03:12 (14.4 MB/s) - ‘syn.int’ saved [23144/23144]

Normal data size: (2398, 2)
--2022-04-04 23:03:12--  https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/3.Synthetic-lpr/intrusion/exploit-unm.int
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sen

**Change to covert all syscall of 1 PID into a data record. Pasrse each data record to a length of 10 or 15, clean frequent records.**

In [80]:
df.shape, intrusiondf.shape

((2398, 2), (164232, 2))

**Create syscall sequence per pid**

In [81]:
# This function groups data by PID, so the sequences appear by PID instead of by order, in case it was interrupted by other PID
## It returns a dict with PID as key and syscall seq as item
def group_syscalls_by_pid (data):
  seq_per_pid = {}
  for p in data['PID'].unique():
    filt = data['PID'] == p
    seq = data.loc[filt]['Syscall'].values.astype(str)
    seq_per_pid[p] = ' '.join(seq)
  return seq_per_pid

if df.shape[1] == 2:
  print('Number of unique PID in normal data:', len(df['PID'].value_counts()))
  print('Number of unique PID in intrusion data:', len(intrusiondf['PID'].value_counts()))

  # Group normal df by PID and drop PID column
  normal_seq_per_pid = group_syscalls_by_pid(df)
  print('Number of unique PID in normal:', len(normal_seq_per_pid))
  #print('Normal PIDs and its sequences: ',normal_seq_per_pid)

  # Do the same thing to intrusion PID
  intrusion_seq_per_pid = group_syscalls_by_pid(intrusiondf)
  print('Number of unique PID in intrusion:', len(intrusion_seq_per_pid))
  #print('Intrusion PIDs and its sequences: ', intrusion_seq_per_pid)

else:
  print('ADFA-LD data does not have PID column')

Number of unique PID in normal data: 9
Number of unique PID in intrusion data: 1001
Number of unique PID in normal: 9
Number of unique PID in intrusion: 1001


In [82]:
# Drop a sequence if its total len is less than 3
def remove_small_seq(pid_seq_dict):
  removed_pid = []
  for pid in pid_seq_dict:
    seq_list = pid_seq_dict[pid].split()
    if len(seq_list) < 3:
      print('Remove PID %d which only has %d syscals in its sequence: %s' % (pid, len(seq_list), pid_seq_dict[pid]))
      removed_pid.append(pid)

  [pid_seq_dict.pop(pid) for pid in removed_pid]
  return pid_seq_dict

if df.shape[1] ==2:
  # Clean small intrusion sequences
  print('Clean small normal seq: \nNum PID in Normal before:', len(normal_seq_per_pid))
  normal_seq_per_pid = remove_small_seq(normal_seq_per_pid)
  df = pd.DataFrame.from_dict(normal_seq_per_pid, orient = 'index', columns=['Syscall Sequence'] )
  print('Num PID in Normal after:', len(normal_seq_per_pid))

  # Clean small intrusion sequences
  print('\n\nClean small intrusion seq: \nNum PID in Intrusion before:', len(intrusion_seq_per_pid))
  intrusion_seq_per_pid = remove_small_seq(intrusion_seq_per_pid)
  intrusiondf = pd.DataFrame.from_dict(intrusion_seq_per_pid, orient = 'index', columns=['Syscall Sequence'] )
  print('Num PID in Intrusion after:', len(intrusion_seq_per_pid))

Clean small normal seq: 
Num PID in Normal before: 9
Num PID in Normal after: 9


Clean small intrusion seq: 
Num PID in Intrusion before: 1001
Num PID in Intrusion after: 1001


## **Data Parsing**

In [83]:
df

,Syscall Sequence
1524,4 2 66 66 4 138 66 5 23 45 4 27 66 5 4 2 66 66...
1530,4 2 66 66 4 138 66 5 23 45 4 27 66 5 4 2 66 66...
1537,4 2 66 66 4 138 66 5 23 45 4 27 66 5 4 2 66 66...
1544,4 2 66 66 4 138 66 5 23 45 4 27 66 5 4 2 66 66...
1551,4 2 66 66 4 138 66 5 23 45 4 27 66 5 4 2 66 66...
1558,4 2 66 66 4 138 66 5 23 45 4 27 66 5 4 2 66 66...
1565,4 2 66 66 4 138 66 5 23 45 4 27 66 5 4 2 66 66...
1571,4 2 66 66 4 138 66 5 23 45 4 27 66 5 4 2 66 66...
1577,4 2 66 66 4 138 66 5 23 45 4 27 66 5 4 2 66 66...


In [84]:
## Parse an entire Syscall seq per PID into smaller sequences of size 15
def parse_seq(seq_per_pid):
  sequences = pd.DataFrame()
  for _, row in seq_per_pid.iterrows():
    token = word_tokenize(row['Syscall Sequence'])  # Tokenize the string of sequence
 
    # Parse the sequence into length of 15
    sequences=sequences.append(list(nltk.ngrams(token, 25, pad_right=True, right_pad_symbol=-1)))
    #print('PID %d - seq len: %d'% (p, len(sequences)))
  return sequences

#normal = parse_seq(normal_seq_per_pid)
normal = parse_seq(df)
normal

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,4,2,66,66,4,138,66,5,23,45,...,2,66,66,66,5,4,2,66,66,5
1,2,66,66,4,138,66,5,23,45,4,...,66,66,66,5,4,2,66,66,5,5
2,66,66,4,138,66,5,23,45,4,27,...,66,66,5,4,2,66,66,5,5,105
3,66,4,138,66,5,23,45,4,27,66,...,66,5,4,2,66,66,5,5,105,104
4,4,138,66,5,23,45,4,27,66,5,...,5,4,2,66,66,5,5,105,104,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,9,5,3,5,5,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
163,5,3,5,5,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
164,3,5,5,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
165,5,5,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [85]:
print('Parsing Intrusion')
intrusion = parse_seq(intrusiondf)
intrusion

Parsing Intrusion


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,4,2,66,66,4,138,66,5,23,45,...,2,66,66,66,5,4,2,66,66,5
1,2,66,66,4,138,66,5,23,45,4,...,66,66,66,5,4,2,66,66,5,5
2,66,66,4,138,66,5,23,45,4,27,...,66,66,5,4,2,66,66,5,5,105
3,66,4,138,66,5,23,45,4,27,66,...,66,5,4,2,66,66,5,5,105,104
4,4,138,66,5,23,45,4,27,66,5,...,5,4,2,66,66,5,5,105,104,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,9,9,5,5,5,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
184,9,5,5,5,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
185,5,5,5,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
186,5,5,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


Start tokenizing system calls into n-grams as specified in the params at the top of the page.

## **Preparing data for data partition**
- Combine data and labels together. 
- Combine data from each class together --> Create a pool of data 
- Split data pool into train and test

## **Cleaning**
- Remove dupplication between train and test
- Save train set. Save test set as unclean test
- Clean test set and save as clean test 

- 2 directions for Testing set:
  *  Test with clean data: Remove overlap and dupplication between Normal and Intrusion in Test data
  *  Test with Unclean data: Leave Test data as is

- 2 directions Training set:
  *   Clean model: Remove overlap and dupplication between Normal and Intrusion in Train data
  *   Unclean model: Leave Train data as is



**Add Label to Data**

In [86]:
print('Normal len:',len(normal),'\nIntrusion len:', len(intrusion))
normal['Label'] = 0     # Normal sequences is labeled 0.
intrusion['Label'] = 1  # Intrusion sequences is labeled 1.

Normal len: 2398 
Intrusion len: 164232


## **Partition Training and Testing dataset 70/30**
- Combine both classes together then split data.
- We do not need to bootstrap since there are more than enough data from both classes.


In [87]:
# combine normal(train) and intrusion(test) data and split them into training and testing sets
df = normal.append(intrusion, ignore_index=True).astype(int)
print('Df sz:', df.shape)

# Spliting into training and testing
x_train, x_test, y_train, y_test = train_test_split(df, df['Label'], test_size = 0.30, shuffle=True, stratify=df['Label'])

# Reset index of training and testing sets
x_train.reset_index(drop=True, inplace=True); y_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True);  y_test.reset_index(drop=True, inplace=True)

print('Train sz:',len(x_train), len(y_train))
intrusion_train = y_train.loc[y_train == 1]
normal_train = y_train.loc[y_train == 0]
print('Train set: Intrusion vs. Normal cases', len(y_train.iloc[intrusion_train] ), len(y_train.iloc[normal_train] ))

print('Test sz:', len(x_test), len(y_test))
intrusion_test = y_test.loc[y_test == 1]
normal_test = y_test.loc[y_test == 0]
print('Test set: Intrusion vs. Normal cases', len(y_test.iloc[intrusion_test] ), len(y_test.iloc[normal_test] ))

Df sz: (166630, 26)
Train sz: 116641 116641
Train set: Intrusion vs. Normal cases 114962 1679
Test sz: 49989 49989
Test set: Intrusion vs. Normal cases 49270 719


## **Data Cleaning**
**Remove Overlap between Train and Test**
--> Ensures reliability in model performance

**Remove Dupplicate or Frequent Data in Test""
--> Ensures a fair evaluation because if Test has a lot of dupplication, model will either classify all dupplication correctly or incorrectly --> Not fair and not reliable in evaluating model performance 

### **Cleaning testing data**

In [88]:
## Get % of duplicates in both datasets

# Convert normal df to set, and intrusion df to set
def clean_data(normal, intrusion):
  normal_list = normal.values.tolist()
  intrusion_list = intrusion.values.tolist()
  normal_set = set(tuple(i) for i in normal_list)
  intrusion_set = set(tuple(i) for i in intrusion_list)
  print('List sz vs. Set sz of normal sequences: %d vs. %d'% (len(normal_list),len(normal_set)) )
  print('List sz vs. Set sz of intrusion sequences: %d vs. %d'% (len(intrusion_list),len(intrusion_set)) )
    
  normal_dupplication = (len(normal_list) - len(normal_set)) /len(normal_list)*100 
  intrusion_duplication = (len(intrusion_list)-len(intrusion_set))/len(intrusion_list) * 100

  print('Duplication Rate in Normal Class: %.3f%%'% normal_dupplication )
  print('Duplication Rate in Intrusion Class: %.3f%%'% intrusion_duplication) 
 
  c_intrusion = intrusion_set - normal_set 
  overlap_rate =  len(normal_set.intersection(intrusion_set)) / len(normal_set.union(intrusion_set)) * 100
  print('Overlap rate: %.3f%%' % overlap_rate)
  
  #c_normal = normal_set - intrusion_set
  if len(c_intrusion) == 0:
    print(DATA+' No Duplication!')
  if len(c_intrusion) > 0:
    intrusion = pd.DataFrame(c_intrusion)
  else:
    intrusion = pd.DataFrame(intrusion_set)
  #if len(c_normal) > 0:
  #  normal = pd.DataFrame(c_normal)
  #else:
  normal = pd.DataFrame(normal_set)

  print('After cleaning: \nNormal sz:', len(normal), ' Intrusion sz:', len(c_intrusion) )
  return normal, intrusion


def remove_overlap2(train, test):
  train_list = train.values.tolist()
  test_list = test.values.tolist()
  train_set = set(tuple(i) for i in train_list)
  test_set = set(tuple(i) for i in test_list)
  print('List sz vs. Set sz of training sequences: %d vs. %d'% (len(train_list),len(train_set)) )
  print('List sz vs. Set sz of testing sequences: %d vs. %d'% (len(test_list),len(test_set)) )
    
  train_dupplication  = (len(train_list) - len(train_set)) /len(train_list)*100 
  test_duplication    = (len(test_list) - len(test_set))/len(test_list) * 100

  print('Duplication Rate in training set: %.3f%%'% train_dupplication )
  print('Duplication Rate in test set: %.3f%%'% test_duplication) 

  intersect = train_set.intersection(test_set)
  overlap_rate = len(intersect)*100/(len(train_set.union(test_set)))
  print( 'Overlap rate is %.3f%%'% overlap_rate)
  intersection_df = pd.DataFrame.from_dict(intersect).rename(columns={25:'Label'})
  independent_test = pd.merge(intersection_df, test, how = 'outer', indicator = True).query('_merge=="right_only"').drop(columns = '_merge')

  return independent_test

In [89]:
independent_test = remove_overlap2(x_train, x_test)
independent_test

List sz vs. Set sz of training sequences: 116641 vs. 617
List sz vs. Set sz of testing sequences: 49989 vs. 507
Duplication Rate in training set: 99.471%
Duplication Rate in test set: 98.986%
Overlap rate is 60.342%


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,Label
49897,4,127,2,18,3,5,56,7,56,119,...,32,17,4,2,5,4,56,7,56,1
49898,121,112,5,17,4,127,2,18,3,5,...,3,3,3,3,32,17,4,2,5,1
49899,121,112,5,17,4,127,2,18,3,5,...,3,3,3,3,32,17,4,2,5,1
49900,89,121,112,5,17,4,127,2,18,3,...,3,3,3,3,3,32,17,4,2,1
49901,89,121,112,5,17,4,127,2,18,3,...,3,3,3,3,3,32,17,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49984,5,56,7,56,119,119,3,3,3,3,...,53,3,3,3,3,5,4,2,18,1
49985,3,3,5,5,4,2,18,3,5,8,...,5,3,5,5,-1,-1,-1,-1,-1,0
49986,112,5,17,4,127,2,18,3,5,56,...,3,3,3,32,17,4,2,5,53,1
49987,5,56,7,56,119,119,3,3,3,3,...,4,56,7,56,119,2,3,2,5,1


In [90]:
x_test

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,Label
0,66,66,4,138,66,5,23,45,4,27,...,66,5,4,2,66,66,5,5,105,1
1,104,106,105,104,104,106,105,104,104,106,...,104,106,105,104,104,106,105,104,104,1
2,2,5,50,27,4,27,88,167,167,167,...,5,4,18,2,5,155,4,50,27,1
3,66,66,5,5,105,104,104,106,105,104,...,105,104,104,106,105,104,104,106,105,1
4,105,104,104,106,105,104,104,106,105,104,...,105,104,104,106,105,104,104,106,105,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49984,5,23,45,4,27,66,5,4,2,66,...,66,5,5,105,104,104,106,105,104,1
49985,105,104,104,106,105,104,104,106,105,104,...,105,104,104,106,105,104,104,106,105,1
49986,5,4,18,2,5,155,4,50,27,2,...,100,5,112,19,93,19,100,50,88,1
49987,106,105,104,104,106,105,104,104,106,9,...,5,-1,-1,-1,-1,-1,-1,-1,-1,1


In [91]:
# Check how much data have left after condensing it as a set
a = independent_test.values.tolist()
a_set = set(tuple(i) for i in a)
len(a_set)

84

left = pd.DataFrame({'ID': [1,2,3,1,4], 'name': ['Ngan', 'Juan', 'Dave', 'Ngan', 'JD']})
right = pd.DataFrame({'ID': [2,2,4], 'name': ['Natti', 'Natti', 'JD']})

pd.merge(left, right, how='outer', indicator=True).query('_merge=="left_only"')

In [92]:
# Separate normal and intrusion in Test Clean so that I can call func clean_data on them
def separate_two_classes (data):
  ## Filter normal data from Test and drop label column
  filt = data.loc[:, 'Label'] == 0
  normal_class = data.loc[filt]
  normal_class.drop(columns = 'Label', inplace = True)

  ## Filter Intrusion data from Test and drop label column
  intrusion_class = data.loc[~filt]
  intrusion_class.drop(columns = 'Label', inplace = True)
  return normal_class, intrusion_class

x_test = independent_test.copy()
test_normal, test_intrusion = separate_two_classes(x_test)
print('Before cleaning:', x_test.shape)
test_clean_normal, test_clean_intrusion = clean_data(test_normal, test_intrusion)

Before cleaning: (92, 26)
List sz vs. Set sz of normal sequences: 32 vs. 28
List sz vs. Set sz of intrusion sequences: 60 vs. 56
Duplication Rate in Normal Class: 12.500%
Duplication Rate in Intrusion Class: 6.667%
Overlap rate: 0.000%
After cleaning: 
Normal sz: 28  Intrusion sz: 56


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [93]:
# Add label back to data
test_clean_normal['Label'] = 0
test_clean_intrusion['Label'] = 1
test_clean = test_clean_normal.append(test_clean_intrusion, ignore_index= True)
test_clean = test_clean.sample(frac=1).reset_index(drop=True) #Shuffle data
test_clean

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,Label
0,19,100,100,100,100,50,88,100,143,128,...,5,17,4,127,2,18,3,5,56,1
1,2,18,3,5,56,7,56,119,119,3,...,4,2,5,4,56,7,56,119,2,1
2,127,66,5,93,100,5,112,19,93,19,...,100,143,128,85,89,128,89,121,112,1
3,56,7,56,119,2,3,2,5,3,3,...,3,5,8,50,3,105,104,104,106,1
4,119,119,3,3,3,3,3,32,17,4,...,88,27,167,5,50,27,4,56,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,3,3,3,5,4,2,18,3,5,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
80,4,2,5,53,3,3,3,3,5,4,...,5,5,5,-1,-1,-1,-1,-1,-1,1
81,4,2,5,4,56,7,56,119,2,3,...,5,4,2,18,3,5,8,50,3,1
82,56,7,56,119,119,3,3,3,3,3,...,17,4,27,88,27,167,5,50,27,0


In [94]:
test_clean.shape, x_test.shape

((84, 26), (92, 26))

###**Cleaning training data**

In [95]:
train_normal, train_intrusion = separate_two_classes(x_train)
print('Before cleaning:', x_train.shape)
train_clean_normal, train_clean_intrusion = clean_data(train_normal, train_intrusion)

# Add label back to data
train_clean_normal['Label'] = 0
train_clean_intrusion['Label'] = 1
train_clean = pd.concat([train_clean_normal, train_clean_intrusion], ignore_index=True) #test_clean_normal.append(test_clean_intrusion, ignore_index= True)
train_clean = train_clean.sample(frac=1).reset_index(drop=True) #Shuffle data
print('After cleaning:', len(train_clean))

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Before cleaning: (116641, 26)
List sz vs. Set sz of normal sequences: 1679 vs. 303
List sz vs. Set sz of intrusion sequences: 114962 vs. 314
Duplication Rate in Normal Class: 81.954%
Duplication Rate in Intrusion Class: 99.727%
Overlap rate: 28.542%
After cleaning: 
Normal sz: 303  Intrusion sz: 177
After cleaning: 480


In [96]:
x_train.to_csv('train.csv', index=False)
train_clean.to_csv('train_clean.csv', index=False)
test_clean.to_csv('test_clean.csv', index=False)
x_test.to_csv('test_unclean.csv', index=False)

In [97]:
x_train.to_csv("train.csv.gz", index=False, compression="gzip")